In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_iris  
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score

import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

from sklearn.model_selection import KFold

In [2]:
# 모델링 - 회귀(Regression) - 불량율

test_df_3 = pd.read_csv("./test_df_3.csv")
a = test_df_3.sort_values(by='불량율_2', ascending = True)
a = a.reset_index(drop='True')
a

,Unnamed: 0,Humidity_max,Temperature_mean,Humidity_mean,case_1,불량율(%),불량율_2
0,24,52.005,33.158153,6.818484,case 31,4.2,0.042
1,47,52.009,33.158146,6.818470,case 52,4.2,0.042
2,31,52.001,33.158050,6.818792,case 38,4.2,0.042
3,16,51.998,33.158143,6.818524,case 24,4.2,0.042
4,39,51.998,33.158136,6.818560,case 45,4.2,0.042
5,23,52.005,33.157015,6.821096,case 30,4.6,0.046
6,46,52.009,33.157007,6.821092,case 51,4.6,0.046
7,38,51.998,33.157004,6.821142,case 44,4.6,0.046
8,30,52.001,33.156906,6.821454,case 37,4.6,0.046
9,15,51.998,33.157008,6.821119,case 23,4.6,0.046


In [3]:
# 데이터 늘리기 
# 동일한 균형으로 데이터 생성

# 난수 생성 - label(불량율)
import random

target_list = []
for i in range(145):
    value = round(random.uniform(0.03,0.3),4)
    # print(value)
    target_list.append(value)
    
len(target_list)
# feature 값 생성하기 target_list

145

In [4]:
# feature 값 생성하기 
# 구간 함수 생성

def Humi_mean(t):
    if t>=0.2 and t<0.3:
        H = 10*t + 4.8
    else:
        H = 6.8
        
    return H

def Temp_mean(t):
    if t>=0.2 and t<0.3:
        T = -19.4*t + 37
    else: 
        T = 33.15  # 평균 값으로 대체 
    
    return T

def Humi_max(t):
    if t<=0.072:
        H_m = 52
    elif t>0.072 and t<0.08:
        H_m = 53
    elif t == 0.08:
        H_m = 54.5  # 54~55.1 
    elif t>0.08 and t<0.082:
        H_m = 53
    elif t>=0.082 and t<=0.097:
        H_m = 52 #51.99~52
    elif t>0.097 and t<0.3:
        H_m = -19*t + 58.2

    return H_m

In [5]:
Humi_mean_l = []
Temp_mean_l = []
Humi_max_l = []


for t in target_list:
    #print(t)
    H = Humi_mean(t)
    T = Temp_mean(t)
    H_m = Humi_max(t)

    Humi_mean_l.append(H)
    Temp_mean_l.append(T)
    Humi_max_l.append(H_m)
    
    
df = pd.DataFrame()
df['Humidity_mean'] = Humi_mean_l
df['Temperature_mean'] = Temp_mean_l
df['Humidity_max'] = Humi_max_l
df['불량율'] = target_list
#
p = [round(i,2)*100 for i in target_list]
df['불량율(%)'] = p
print(df)
print(df.describe())


     Humidity_mean  Temperature_mean  Humidity_max     불량율  불량율(%)
0            6.800          33.15000       55.5590  0.1390    14.0
1            7.704          31.36624       52.6824  0.2904    29.0
2            6.800          33.15000       52.0000  0.0440     4.0
3            6.800          33.15000       56.2791  0.1011    10.0
4            6.800          33.15000       55.6027  0.1367    14.0
..             ...               ...           ...     ...     ...
140          6.800          33.15000       54.7021  0.1841    18.0
141          6.800          33.15000       56.1499  0.1079    11.0
142          6.800          33.15000       52.0000  0.0840     8.0
143          6.800          33.15000       56.0948  0.1108    11.0
144          6.800          33.15000       52.0000  0.0717     7.0

[145 rows x 5 columns]
       Humidity_mean  Temperature_mean  Humidity_max         불량율      불량율(%)
count     145.000000        145.000000    145.000000  145.000000  145.000000
mean        6.9637

In [8]:
test_df = pd.concat([df[['Humidity_mean','Temperature_mean','Humidity_max','불량율(%)']],a[['Humidity_mean','Temperature_mean','Humidity_max','불량율(%)']]])
test_df = test_df.reset_index()
test_df

,index,Humidity_mean,Temperature_mean,Humidity_max,불량율(%)
0,0,6.800000,33.150000,55.5590,14.0
1,1,7.704000,31.366240,52.6824,29.0
2,2,6.800000,33.150000,52.0000,4.0
3,3,6.800000,33.150000,56.2791,10.0
4,4,6.800000,33.150000,55.6027,14.0
...,...,...,...,...,...
195,50,7.812161,31.167189,52.5280,29.3
196,51,7.812510,31.167095,52.6160,29.3
197,52,7.812100,31.167189,52.4820,29.3
198,53,7.812135,31.167188,52.5130,29.3


In [9]:
target_yn = []
for i  in test_df['불량율(%)']:
    if i>10:
        target_yn.append("y") #불량
    else:
        target_yn.append("n") #정상
        
test_df['target_yn'] = target_yn
test_df.head()

# LabelEncoder를 객체로 생성
encoder = LabelEncoder()
# fit, transform 메소드를 통한 레이블 인코딩
encoder.fit(test_df['target_yn'])
test_df["target"] = encoder.transform(test_df['target_yn'])
print(test_df.head())

   index  Humidity_mean  Temperature_mean  Humidity_max  불량율(%) target_yn  \
0      0          6.800          33.15000       55.5590    14.0         y   
1      1          7.704          31.36624       52.6824    29.0         y   
2      2          6.800          33.15000       52.0000     4.0         n   
3      3          6.800          33.15000       56.2791    10.0         n   
4      4          6.800          33.15000       55.6027    14.0         y   

   target  
0       1  
1       1  
2       0  
3       0  
4       1  


In [11]:
#train_test_split
x_train, x_test, y_train, y_test = train_test_split(test_df[['Humidity_max','Temperature_mean','Humidity_mean','target']], test_df['target'].values, test_size=0.5, random_state=7777)
print("X_train:", x_train.head())
print("y_train:", y_train)

X_train:      Humidity_max  Temperature_mean  Humidity_mean  target
51        52.0000         33.150000       6.800000       0
5         53.7540         32.460400       7.140000       1
3         56.2791         33.150000       6.800000       0
167       54.5680         33.149628       6.841089       0
155       52.0920         33.155840       6.823909       0
y_train: [0 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1
 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1]


In [12]:
dt = DecisionTreeClassifier(random_state = 20)
dt.fit(x_train,y_train)
pred = dt.predict(x_test)
print(pred)
print(y_test)

[1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0
 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1]
[1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0
 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1]


In [13]:
print('accuracy:{0:.2f}'.format(accuracy_score(y_test,pred)))

accuracy:1.00


In [14]:
result = pd.DataFrame()
result['y_test'] = y_test
result['pred'] = pred
result

,y_test,pred
0,1,1
1,0,0
2,1,1
3,0,0
4,0,0
...,...,...
95,1,1
96,0,0
97,0,0
98,0,0


In [15]:
confusion_matrix = confusion_matrix(result['y_test'],result['pred'])
confusion_matrix = pd.DataFrame(confusion_matrix, index=['실제값(TN)', '실제값(FP)'], columns=['예측값(FN)', '예측값(TP)'])
confusion_matrix


,예측값(FN),예측값(TP)
실제값(TN),34,0
실제값(FP),0,66


In [16]:
print(classification_report(result['y_test'], result['pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        66

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [17]:
# 10-fold cross validation(classification)
# 높은 정확도로 우연한 결과인지 판단하기 위해 k-flod 교차검증을 통해 평균 accuracy를 구한다.

n = 0
total_acc = []
kf = KFold(n_splits=2)
feature = test_df[['Humidity_max','Temperature_mean','Humidity_mean']]
label = test_df['target']

for train_idx, test_idx in kf.split(feature):
    print("train_data: %s, test_data: %s" % (train_idx, test_idx))
    print(len(train_idx),len(test_idx))
    x_train, x_test = feature.iloc[train_idx,], feature.iloc[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    dt = DecisionTreeClassifier()
    dt.fit(x_train,y_train)
    pred = dt.predict(x_test)
    n += 1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    
    print("\n # {0}번째 교차 검증 정확도: {1}".format(n,accuracy))
    print(classification_report(y_test, pred))
    total_acc.append(accuracy)
    
print("\n 평균 정확도: ", np.mean(total_acc))

train_data: [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199], test_data: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
100 100

 # 1번째 교차 검증 정확도: 0.97
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        23
           1       0.96      1.00      0.98        77

    accuracy                           0.97       100
   macro avg 